## 06_2. Epithelial Cells  -- Align cells along differentiation trajectories

<div style="text-align: left;">
    <p style="text-align: left;">Updated Time: 2025-02-13</p>
</div>

##### Load libraries

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import scanpy as sc
import omicverse as ov
from omicverse.externel import VIA

import matplotlib.pyplot as plt
ov.plot_set()

import warnings
warnings.simplefilter("ignore")

##### Set working directory for analysis

In [ ]:
cwd = '/media/bio/Disk/Research Data/EBV/omicverse'
os.chdir(cwd)
updated_dir = os.getcwd()
print("Updated working directory: ", updated_dir)

#### Reading in annotated AnnData object

In [ ]:
adata_epi = sc.read_h5ad("Processed Data/scRNA_Epi_CNV.h5ad")
adata_epi

In [ ]:
print(np.min(adata_epi.X), np.max(adata_epi.X))

In [ ]:
for i in adata_epi.obs['Epi_celltype'].cat.categories:
  number = len(adata_epi.obs[adata_epi.obs['Epi_celltype']==i])
  print('the number of category {} is {}'.format(i,number))

You can use `recover_counts` to recover the raw counts after normalize and log1p

In [ ]:
X_counts_recovered, size_factors_sub=ov.pp.recover_counts(adata_epi.X, 50*1e4, 50*1e5, log_base=None, chunk_size=50000)
adata_epi.layers['counts']=X_counts_recovered

In [ ]:
adata_epi.layers['counts'].shape
print(np.min(adata_epi.layers['counts']), np.max(adata_epi.layers['counts']))

In [ ]:
adata_epi.X=adata_epi.layers['counts']
print(np.min(adata_epi.X), np.max(adata_epi.X))

##### Save the .raw attribute of the AnnData object

Set the .raw attribute of the AnnData object to the raw gene expression (counts) for later use. This simply freezes the state of the AnnData object.
You can get back an AnnData of the object in .raw by calling .raw.to_adata().

In [ ]:
adata_epi.raw = adata_epi.copy() # This saves the raw count data in adata.raw

##### Preprocess data


In [ ]:
adata_epi=ov.pp.preprocess(adata_epi,mode='shiftlog|pearson',n_HVGs=2000)
adata_epi = adata_epi[:, adata_epi.var.highly_variable_features]
ov.pp.scale(adata_epi)
ov.pp.pca(adata_epi,layer='scaled',n_pcs=50)

Let us inspect the contribution of single PCs to the total variance in the data. 
This gives us information about how many PCs we should consider in order to compute the neighborhood relations of cells. In our experience, often a rough estimate of the number of PCs does fine.


In [ ]:
ov.utils.plot_pca_variance_ratio(adata_epi)

#### Unsupervised clustering
The Leiden algorithm is as an improved version of the Louvain algorithm which outperformed other clustering methods for single-cell RNA-seq data analysis ([Du et al., 2018, Freytag et al., 2018, Weber and Robinson, 2016]). Since the Louvain algorithm is no longer maintained, using Leiden instead is preferred.

We, therefore, propose to use the Leiden algorithm[Traag et al., 2019] on single-cell k-nearest-neighbour (KNN) graphs to cluster single-cell datasets.

Leiden creates clusters by taking into account the number of links between cells in a cluster versus the overall expected number of links in the dataset.

Here, we set method='leiden' to cluster the cells using Leiden

In [ ]:
ov.pp.neighbors(adata_epi, n_neighbors=15, n_pcs=20,use_rep='scaled|original|X_pca')
ov.pp.umap(adata_epi)

In [ ]:
ov.utils.embedding(adata_epi,basis='X_umap',
                   color=['Epi_celltype'],
                   frameon='small',wspace=0.5)

### Trajectory Inference with StaVIA
VIA is a single-cell Trajectory Inference method that offers topology construction, pseudotimes, automated terminal state prediction and automated plotting of temporal gene dynamics along lineages. Here, we have improved the original author's colouring logic and user habits so that users can use the anndata object directly for analysis。

We have completed this tutorial using the analysis provided by the original VIA authors.

Paper: Generalized and scalable trajectory inference in single-cell omics data with VIA

Code: https://github.com/ShobiStassen/VIA

Colab_Reproducibility：https://colab.research.google.com/drive/1A2X23z_RLJaYLbXaiCbZa-fjNbuGACrD?usp=sharing

##### Model construct and run
We need to specify the cell feature vector use_rep used for VIA inference, which can be X_pca, X_scVI or X_glue, depending on the purpose of your analysis, here we use X_pca directly. We also need to specify how many components to be used, the components should not larger than the length of vector.

Besides, we need to specify the clusters to be colored and calculate for VIA. If the root_user is None, it will be calculated the root cell automatically.

We need to set basis argument stored in adata.obsm. An example setting tsne because it stored in obsm: 'tsne', 'MAGIC_imputed_data', 'palantir_branch_probs', 'X_pca'

We also need to set clusters argument stored in adata.obs. It means the celltype key.

Other explaination of argument and attributes could be found at https://pyvia.readthedocs.io/en/latest/notebooks/ViaJupyter_scRNA_Hematopoiesis.html

StaVia for time-series

Via 2.0 Cartography on Mouse Gastrulation
Via 2.0 Cartography on Zebrahub (Trajectory Inference)
Via 2.0 Cartography on Zebrahub (Visualization)
StaVia for spatial-temporal

Stavia Spatially aware cartography on MERFISH
StaVia TI for Spatio-temporal Analysis of single cell data

##### Initialize and Run the Via2.0 Class

In [ ]:
ncomps=20
knn=15
v0_random_seed=42
root_user = ['Normal ECs'] #the index of a cell belonging to the nIPC cell type
memory = 50
dataset = 'Epi_celltype'
use_rep = 'scaled|original|X_pca'
clusters = 'Epi_celltype'
basis='X_umap'

'''
#NOTE1, if you decide to choose a cell type as a root, then you need to set the dataset as 'group'
#root_user=['HSC1']
#dataset = 'group'# 'humanCD34'
#NOTE2, if rna-velocity is available, considering using it to compute the root automatically- see RNA velocity tutorial
'''

v0 = VIA.core.VIA(data=adata_epi.obsm[use_rep][:, 0:ncomps], 
             true_label=adata_epi.obs[clusters], 
             edgepruning_clustering_resolution=0.15, cluster_graph_pruning=0.15,
             num_threads=1, #single thread gurrantee the pseudotime reproducibility
             knn=knn,  root_user=root_user, resolution_parameter=1.5,
             dataset=dataset, random_seed=v0_random_seed, memory=memory)#, do_compute_embedding=True, embedding_type='via-atlas')

v0.run_VIA()

In [ ]:
fig, ax, ax1 = VIA.core.plot_piechart_viagraph_ov(adata_epi, clusters='Epi_celltype', dpi=150,
                                             via_object=v0, ax_text=False,show_legend=False)
fig.set_size_inches(8,4)

In [ ]:
adata_epi.obs['pt_via']=v0.single_cell_pt_markov
ov.pl.embedding(adata_epi,basis='X_umap', color=['pt_via'], frameon=True, cmap='Reds')

In [ ]:
# plot the pt_via trajectory
mask = np.in1d(adata_epi.obs["Epi_celltype"], 
               ['Normal ECs', 'EBV- ECs', 'EBV+ ECs', 'EBV- CCs', 'EBV+ CCs'])

plt.figure(figsize=(4, 3))
sc.pl.violin(
    adata_epi[mask],
    keys=["pt_via"],
    groupby="Epi_celltype",
    rotation=-90,
    jitter=False,
    show=False,
    order=['Normal ECs', 'EBV- ECs', 'EBV+ ECs', 'EBV- CCs', 'EBV+ CCs'],
)

plt.savefig('Results/06.Epithelial/06.pt_via_pseudotime_violinplots.pdf', format='pdf')
plt.show()


In [ ]:
clusters='Epi_celltype'
color_true_list=adata_epi.uns['{}_colors'.format(clusters)]

In [ ]:
import numpy as np
np.int = np.int64


In [ ]:
fig, ax, ax1 = VIA.core.plot_trajectory_curves_ov(adata_epi, clusters='Epi_celltype', dpi=150,
                                                  via_object=v0, embedding=adata_epi.obsm['X_umap'],
                                                  draw_all_curves=False)


In [ ]:
v0.embedding = adata_epi.obsm['X_umap']
fig, ax = VIA.core.plot_atlas_view(via_object=v0, 
                                   n_milestones=150, 
                                   sc_labels=adata_epi.obs['Epi_celltype'], 
                                   fontsize_title=12,
                                   fontsize_labels=12,dpi=80,
                                   extra_title_text='Atlas View colored by pseudotime')
fig.set_size_inches(4,4)

In [ ]:
# edge plots can be made with different edge resolutions. Run hammerbundle_milestone_dict() to recompute the edges for plotting. Then provide the new hammerbundle as a parameter to plot_edge_bundle()
# it is better to compute the edges and save them to the via_object. this gives more control to the merging of edges
decay = 0.6 #increasing decay increasing merging of edges
i_bw = 0.02 #increasing bw increases merging of edges
global_visual_pruning = 0.5 #higher number retains more edges
n_milestones = 200

v0.hammerbundle_milestone_dict= VIA.core.make_edgebundle_milestone(via_object=v0, 
                                                              n_milestones=n_milestones, 
                                                              decay=decay, initial_bandwidth=i_bw,
                                                              global_visual_pruning=global_visual_pruning)


In [ ]:
fig, ax = VIA.core.plot_atlas_view(via_object=v0,  
                              add_sc_embedding=True, 
                              sc_labels_expression=adata_epi.obs['Epi_celltype'], 
                              cmap='jet', sc_labels=adata_epi.obs['Epi_celltype'], 
                              text_labels=True, 
                              extra_title_text='Atlas View by Cell type', 
                              fontsize_labels=3,fontsize_title=3,dpi=300
                             )
fig.set_size_inches(5,4)

In [ ]:
# via_streamplot() requires you to either provide an ndarray as embedding as an input parameter OR for via to have an embedding attribute
fig, ax = VIA.core.via_streamplot_ov(adata_epi,'Epi_celltype',
                                     v0, embedding=adata_epi.obsm['X_umap'], dpi=150,
                             density_grid=0.8, scatter_size=30, 
                             scatter_alpha=0.3, linewidth=0.5)
fig.set_size_inches(5,5)

In [ ]:
#Colored by pseudotime

fig, ax = VIA.core.via_streamplot_ov(adata_epi,'Epi_celltype',
                             v0,density_grid=0.8, scatter_size=30, color_scheme='time', linewidth=0.5, 
                             min_mass = 1, cutoff_perc = 5, scatter_alpha=0.3, marker_edgewidth=0.1, 
                             density_stream = 2, smooth_transition=1, smooth_grid=0.5,dpi=150,)
fig.set_size_inches(5,5)

In [ ]:
v0.terminal_clusters

In [ ]:
fig, axs= VIA.core.plot_sc_lineage_probability(via_object=v0, dpi=150,
                                          marker_lineages=v0.terminal_clusters,
                                          embedding=adata_epi.obsm['X_umap']) #marker_lineages=v0.terminal_clusters to plot all
fig.set_size_inches(12,18)

In [ ]:
fig, axs= VIA.core.plot_atlas_view(via_object=v0, dpi=150,
                              lineage_pathway=v0.terminal_clusters,
                                 fontsize_title = 12,
                                 fontsize_labels = 12,
                             ) #marker_lineages=v0.terminal_clusters to plot all
fig.set_size_inches(12,18)

In [ ]:
gene_list_magic =['MYC','SOX2','CD44','ALDH1A1']
df = adata_epi.to_df()
df_magic = v0.do_impute(df, magic_steps=3, gene_list=gene_list_magic) #optional

In [ ]:
fig, axs = VIA.core.plot_viagraph(via_object=v0, 
                                  type_data='gene',
                                  df_genes=df_magic, 
                                  gene_list=gene_list_magic[0:4], arrow_head=0.1)
fig.set_size_inches(15,4)

In [ ]:
fig, axs = VIA.core.get_gene_expression(via_object=v0, 
                                         marker_lineages=v0.terminal_clusters,
                                         gene_exp=df_magic[gene_list_magic],
                                         )

ax = axs[3]  

legend = ax.legend(loc='upper center', bbox_to_anchor=(-1.35, 1.45), fontsize=8, ncol=6)

fig.set_size_inches(12, 4)
plt.tight_layout()

plt.savefig('Results/06.Epithelial/06.Epithelial_VIA_gene_expression.pdf', format='pdf', bbox_inches='tight')
plt.show()

### Visualization

In [ ]:
fig1, ax1 = plt.subplots(figsize=(4, 4)) 
sc.pl.embedding(adata_epi, basis='X_umap', color='pt_via', frameon=True, cmap='Reds', ax=ax1, show=False)
ax1.set_title('')
for child in ax1.get_children():
    if isinstance(child, plt.cm.ScalarMappable):  
        child.colorbar.remove() 

scatter_obj = ax1.collections[0] 
cbar = fig1.colorbar(scatter_obj, ax=ax1) 
cbar.ax.set_title('VIA Pseudotime', fontsize=10, loc='center')  

umap_file = 'Results/06.Epithelial/UMAP_pt_via_Score.pdf'
plt.savefig(umap_file, dpi=300, bbox_inches='tight')
print(f"UMAP plot saved as: {umap_file}") 
plt.show() 
plt.close()  


via_temp_file = "via_streamplot_temp.png"
fig2, ax2 = plt.subplots(figsize=(4, 4))
VIA.core.via_streamplot_ov(adata_epi, 'Epi_celltype', v0,
                           density_grid=0.8, scatter_size=20, color_scheme='time', 
                           linewidth=0.5, min_mass=1, cutoff_perc=5, scatter_alpha=0.3, 
                           marker_edgewidth=0.1, density_stream=3, smooth_transition=1, 
                           smooth_grid=0.5, dpi=300)
plt.title('')
plt.savefig(via_temp_file, dpi=300, bbox_inches='tight')
plt.close()
via_img = plt.imread(via_temp_file)
ax2.imshow(via_img)
ax2.axis('off')
ax2.set_title('Stream of lineages')
ax2.title.set_position([0.5, -0.2])
os.remove(via_temp_file)

streamplot_file = 'Results/06.Epithelial/VIA_Streamplot.pdf'
plt.savefig(streamplot_file, dpi=300, bbox_inches='tight')
print(f"VIA Streamplot saved as: {streamplot_file}")
plt.show()
plt.close()


fig3, ax3 = plt.subplots(figsize=(4, 4))
ov.pl.bardotplot(adata_epi, groupby='Epi_celltype', color='pt_via', figsize=(4, 4),
                 ax=ax3,
                 ylabel='Expression',
                 bar_kwargs={'alpha': 0.5, 'linewidth': 2, 'width': 0.6, 'capsize': 4},
                 scatter_kwargs={'alpha': 0.8, 's': 10, 'marker': '.'})

ov.pl.add_palue(ax3, line_x1=1, line_x2=2, line_y=0.8,
                text_y=0.02,
                text='$p={}$'.format(round(0.001, 3)),
                fontsize=12, fontcolor='#000000',
                horizontalalignment='center')
ov.pl.add_palue(ax3, line_x1=3, line_x2=4, line_y=0.9,
                text_y=0.02,
                text='$p={}$'.format(round(0.001, 3)),
                fontsize=12, fontcolor='#000000',
                horizontalalignment='center')
ax3.tick_params(axis='x', labelrotation=45)
ax3.set_ylabel('VIA Pseudotime')
ax3.set_title('')
plt.tight_layout(pad=0.1)

violin_file = 'Results/06.Epithelial/pt_via_Score_Violin_Plot.pdf'
plt.savefig(violin_file, dpi=300, bbox_inches='tight')
print(f"Violin plot saved as: {violin_file}")
plt.show()
plt.close()

### Prediction of absolute developmental potential using CytoTrace2
CytoTRACE 2 is a computational method for predicting cellular potency categories and absolute developmental potential from single-cell RNA-sequencing data.

Potency categories in the context of CytoTRACE 2 classify cells based on their developmental potential, ranging from totipotent and pluripotent cells with broad differentiation potential to lineage-restricted oligopotent, multipotent and unipotent cells capable of producing varying numbers of downstream cell types, and finally, differentiated cells, ranging from mature to terminally differentiated phenotypes.

We made three improvements in integrating the CytoTrace2 algorithm in OmicVerse:

No additional packages to install, including R
We fixed a bug in multi-threaded pools to avoid potential error reporting
Native support for anndata, you don't need to export input_file and annotation_file.
If you found this tutorial helpful, please cite CytoTrace2 and OmicVerse:

Kang, M., Armenteros, J. J. A., Gulati, G. S., Gleyzer, R., Avagyan, S., Brown, E. L., Zhang, W., Usmani, A., Earland, N., Wu, Z., Zou, J., Fields, R. C., Chen, D. Y., Chaudhuri, A. A., & Newman, A. M. (2024). Mapping single-cell developmental potential in health and disease with interpretable deep learning. bioRxiv : the preprint server for biology, 2024.03.19.585637. https://doi.org/10.1101/2024.03.19.585637

#### Predict cytotrace2
We need to import the two pre-trained models from CytoTrace2, see the download links for the models:

Figshare: https://figshare.com/ndownloader/files/47258749

or Github: https://github.com/digitalcytometry/cytotrace2/tree/main/cytotrace2_python/cytotrace2_py/resources/17_models_weights https://github.com/digitalcytometry/cytotrace2/tree/main/cytotrace2_python/cytotrace2_py/resources/5_models_weights

All parameters are explained as follows:

adata: AnnData object containing the scRNA-seq data.
use_model_dir: Path to the directory containing the pre-trained model files.
species: The species of the input data. Default is "mouse".
batch_size: The number of cells to process in each batch. Default is 10000.
smooth_batch_size: The number of cells to process in each batch for smoothing. Default is 1000.
disable_parallelization: If True, disable parallel processing. Default is False.
max_cores: Maximum number of CPU cores to use for parallel processing. If None, all available cores will be used. Default is None.
max_pcs: Maximum number of principal components to use. Default is 200.
seed: Random seed for reproducibility. Default is 14.
output_dir: Directory to save the results. Default is 'cytotrace2_results'.

In [ ]:
# The input data for CytoTRACE 2 can be represented as raw or CPM/TPM normalized counts, but should not be log-transformed.
adata_epi = adata_epi.raw.to_adata() # This recovers the raw count data in adata.X
adata_epi.raw = adata_epi.copy() # This saves the raw count data in adata_epi.raw
adata_epi

In [ ]:
print(np.min(adata_epi.X), np.max(adata_epi.X))

In [ ]:
results =  ov.single.cytotrace2(adata_epi,
    use_model_dir="Processed Data/cymodels/17_models_weights",
    species="human",
    batch_size = 10000,
    smooth_batch_size = 1000,
    disable_parallelization = False,
    max_cores = 256,
    max_pcs = 200,
    seed = 14,
    output_dir = 'Results/06.Epithelial/cytotrace2_results'
)

#### Visualizing
Visualizing the results we can directly compare the predicted potency scores with the known developmental stage of the cells, seeing how the predictions meticulously align with the known biology. Take a look!

In [ ]:
ov.utils.embedding(adata_epi,basis='X_umap',
                   color=['Epi_celltype','CytoTRACE2_Score'],
                   frameon='small',cmap='Reds',wspace=0.55)

Left: demonstrates the distribution of different cell types in UMAP space.
Right: demonstrates the CytoTRACE 2 scores of different cell types; cells with high scores are generally considered to have a higher pluripotency or undifferentiated state.

In [ ]:
ov.utils.embedding(adata_epi,basis='X_umap',
                   color=['CytoTRACE2_Potency','CytoTRACE2_Relative'],
                   frameon='small',cmap='Reds',wspace=0.55)

Potency category: The UMAP embedding plot of predicted potency category reflects the discrete classification of cells into potency categories, taking possible values of Differentiated, Unipotent, Oligopotent, Multipotent, Pluripotent, and Totipotent.
Relative order: UMAP embedding of predicted relative order, which is based on absolute predicted potency scores normalized to the range 0 (more differentiated) to 1 (less differentiated). Provides the relative ordering of cells by developmental potential

In [ ]:
fig,ax=plt.subplots(figsize = (4,4))
ov.pl.cellproportion(adata=adata_epi, celltype_clusters='CytoTRACE2_Potency',
                    groupby='Epi_celltype', legend=True,ax=ax)
# 调整图例的字体大小和位置
legend = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10)

In [ ]:
# plot the Erythroid trajectory
mask = np.in1d(adata_epi.obs["Epi_celltype"], 
               ['Normal ECs', 'EBV- ECs', 'EBV+ ECs', 'EBV- CCs', 'EBV+ CCs'])
sc.pl.violin(
    adata_epi[mask],
    keys=["CytoTRACE2_Score","CytoTRACE2_Relative"],
    groupby="Epi_celltype",
    rotation=-90,
    jitter=False,
    order=['Normal ECs', 'EBV- ECs', 'EBV+ ECs', 'EBV- CCs', 'EBV+ CCs'],
    save='_cytotrace2_violinplots.pdf'
)

In [ ]:
fig, ax = plt.subplots(figsize=(4.5, 4)) 
sc.pl.embedding(adata_epi, basis='X_umap', color='CytoTRACE2_Score', frameon=True, cmap='Reds', ax=ax, show=False)
ax.set_title('')  

for child in ax.get_children():
    if isinstance(child, plt.cm.ScalarMappable):
        child.colorbar.remove() 

scatter_obj = ax.collections[0]
cbar = fig.colorbar(scatter_obj, ax=ax)
cbar.ax.set_title('CytoTRACE2 Score', fontsize=10, loc='center')

umap_file = 'Results/06.Epithelial/UMAP_CytoTRACE2_Score.pdf'
plt.savefig(umap_file, dpi=300, bbox_inches='tight')
print(f"UMAP plot saved as: {umap_file}") 
plt.show() 
plt.close()


fig2, ax1 = plt.subplots(figsize=(4.5, 4))
ov.pl.bardotplot(adata_epi, groupby='Epi_celltype', color='CytoTRACE2_Score', figsize=(4, 4), 
                 ax=ax1,
                 ylabel='Expression',
                 bar_kwargs={'alpha': 0.5, 'linewidth': 2, 'width': 0.6, 'capsize': 4},
                 scatter_kwargs={'alpha': 0.8, 's': 10, 'marker': '.'})

ov.pl.add_palue(ax1, line_x1=3, line_x2=4, line_y=0.7,
                text_y=0.02,
                text='$p={}$'.format(round(0.001, 3)),
                fontsize=12, fontcolor='#000000', 
                horizontalalignment='center')
ov.pl.add_palue(ax1, line_x1=1, line_x2=2, line_y=0.6,
                text_y=0.02,
                text='$p={}$'.format(round(0.001, 3)),
                fontsize=12, fontcolor='#000000', 
                horizontalalignment='center')
ax1.tick_params(axis='x', labelrotation=45, labelsize=12) 
ax1.set_ylabel('CytoTRACE2 Score', fontsize=12)
ax1.set_title('') 
plt.tight_layout(pad=0.1)

violin_file = 'Results/06.Epithelial/CytoTRACE2_Score_Violin_Plot.pdf'
plt.savefig(violin_file, dpi=300, bbox_inches='tight')
print(f"Violin plot saved as: {violin_file}")
plt.show() 
plt.close()


fig3, ax2 = plt.subplots(figsize=(5, 4))
ov.pl.cellproportion(adata=adata_epi, celltype_clusters='CytoTRACE2_Potency', groupby='Epi_celltype', 
                     legend=True, ax=ax2)
ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=11) 
ax2.set_title('')  
ax2.set_xlabel('') 
ax2.tick_params(axis='x', labelrotation=45, labelsize=10)
ax2.tick_params(axis='x', labelsize=10)
plt.tight_layout(pad=0.5)

proportion_file = 'Results/06.Epithelial/Cell_Proportion_Plot.pdf'
plt.savefig(proportion_file, dpi=300, bbox_inches='tight')
print(f"Cell proportion plot saved as: {proportion_file}")
plt.show()
plt.close()


from scipy.stats import gaussian_kde
from scipy import stats
pearson_corr, _ = stats.pearsonr(adata_epi.obs["pt_via"], adata_epi.obs["CytoTRACE2_Score"])
print(pearson_corr)

fig4, ax4 = plt.subplots(figsize=(4, 4))
y = adata_epi.obs["pt_via"]
x = adata_epi.obs["CytoTRACE2_Score"]
xy = np.vstack([x, y])
z = gaussian_kde(xy)(xy)
scat = ax4.scatter(x, y, c=z, s=10, cmap="Spectral")

cbar = fig4.colorbar(scat, ax=ax4)  
cbar.ax.set_title('Density', fontsize=10, loc='center') 
ax4.tick_params(axis='both', labelsize=10)
ax4.plot(np.linspace(min(x), max(x), 100), np.linspace(min(y), max(y), 100), color='black', linestyle='--')
ax4.set_ylabel('VIA Pseudotime', fontsize=11)
ax4.set_xlabel('CytoTRACE2 Score', fontsize=11)
ax4.set_title('')

scatter_file = 'Results/06.Epithelial/Scatter_Plot_VIA_Pseudotime_CytoTRACE2_Score.pdf'
plt.savefig(scatter_file, dpi=300, bbox_inches='tight')
print(f"Scatter plot saved as: {scatter_file}")
plt.show()
plt.close()  

#### Save results

In [ ]:
adata_epi

In [ ]:
print(np.min(adata_epi.X), np.max(adata_epi.X))

In [ ]:
adata_epi.write_h5ad("Processed Data/scRNA_Epi_CNV_Traj.h5ad")


**<span style="font-size:16px;">Session information：</span>**

In [ ]:
import sys
import platform
import pkg_resources

# Get Python version information
python_version = sys.version
# Get operating system information
os_info = platform.platform()
# Get system architecture information
architecture = platform.architecture()[0]
# Get CPU information
cpu_info = platform.processor()
# Print Session information
print("Python version:", python_version)
print("Operating system:", os_info)
print("System architecture:", architecture)
print("CPU info:", cpu_info)

# Print imported packages and their versions
print("\nImported packages and their versions:")
for package in pkg_resources.working_set:
    print(package.key, package.version)